In [4]:
import backtest
import pandas as pd
import numpy as np
import df
import sentiment
import joblib
lr=joblib.load('lr.pkl')
vect=joblib.load('vector.pkl')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dhrub\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
C:\Users\dhrub\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
obj2=df.RealDataPipeline(start_date='2024-01-01',end_date='2025-12-31')
obj2.download_stock_prices()
obj2.load_financial_news('apple_news_data.csv')
obj2.align_data()
#print("The merged dataframe is as follows") 
obj2.df_combined.head()

Downloaded 501 days of price data
Loading news data from apple_news_data.csv...
Loaded 29752 news articles for AAPL
Combined dataset shape: (228, 7)
Date range: 2024-01-02 to 2024-11-27


,date,Close,High,Low,Open,Volume,all_headlines
0,2024-01-02,183.903229,186.677036,182.169601,185.399096,82488700,Dow Jones Futures: Tech Stocks Skid To Start 2...
1,2024-01-03,182.526215,184.140970,181.713879,182.496496,58414500,Wall Street: 2024 Is The Year Of The Pivot App...
2,2024-01-04,180.208130,181.377083,179.187767,180.445875,71983600,Early 2024 Markets Sweep Up Excesses Apple Sup...
3,2024-01-05,179.484970,181.050190,178.484425,180.287405,62379700,"Apple’s Losing Streak Hits 4 Days, Amid Report..."
4,2024-01-08,183.823975,183.863609,179.801961,180.386437,59144500,"Can Stocks Surpass 2022 Highs? Yes, but the Ma..."


In [25]:
obj=sentiment.RealDataSentimentAnalyzer(vect,lr)
headlines=obj2.df_combined['all_headlines']
obj2.df_combined[['vader_sentiment','lr_sentiment','finbert_sentiment']]=obj.score_all_headlines(headlines)[['vader_sentiment','lr_sentiment','finbert_sentiment']]
data=obj2.df_combined
data['hold']=0
data.loc[data.index[-1], 'hold'] = -1
data.loc[data.index[0], 'hold'] = 1

FinBERT inference: 100%|█████████████████████████████████████████████████████████████| 228/228 [01:10<00:00,  3.25it/s]


In [26]:
data.head()

,date,Close,High,Low,Open,Volume,all_headlines,vader_sentiment,lr_sentiment,finbert_sentiment,hold
0,2024-01-02,183.903229,186.677036,182.169601,185.399096,82488700,Dow Jones Futures: Tech Stocks Skid To Start 2...,-0.9465,-0.457457,0.014798,1
1,2024-01-03,182.526215,184.140970,181.713879,182.496496,58414500,Wall Street: 2024 Is The Year Of The Pivot App...,0.9464,0.059430,0.079980,0
2,2024-01-04,180.208130,181.377083,179.187767,180.445875,71983600,Early 2024 Markets Sweep Up Excesses Apple Sup...,0.8779,-0.042516,-0.003677,0
3,2024-01-05,179.484970,181.050190,178.484425,180.287405,62379700,"Apple’s Losing Streak Hits 4 Days, Amid Report...",0.9459,-0.529408,0.644689,0
4,2024-01-08,183.823975,183.863609,179.801961,180.386437,59144500,"Can Stocks Surpass 2022 Highs? Yes, but the Ma...",0.9951,-0.015538,0.107775,0


In [27]:
def signal_from_sentiment(sentiment):
    threshold=0.02
    if sentiment > threshold:
        return 1  # Buy
    elif sentiment < -threshold:
        return -1  # Sell
    else:
        return 0  # Hold
data['vader_sentiment']=data['vader_sentiment'].apply(signal_from_sentiment)
data['lr_sentiment']=data['lr_sentiment'].apply(signal_from_sentiment)
data['finbert_sentiment']=data['finbert_sentiment'].apply(signal_from_sentiment)
data.head()

,date,Close,High,Low,Open,Volume,all_headlines,vader_sentiment,lr_sentiment,finbert_sentiment,hold
0,2024-01-02,183.903229,186.677036,182.169601,185.399096,82488700,Dow Jones Futures: Tech Stocks Skid To Start 2...,-1,-1,0,1
1,2024-01-03,182.526215,184.140970,181.713879,182.496496,58414500,Wall Street: 2024 Is The Year Of The Pivot App...,1,1,1,0
2,2024-01-04,180.208130,181.377083,179.187767,180.445875,71983600,Early 2024 Markets Sweep Up Excesses Apple Sup...,1,-1,0,0
3,2024-01-05,179.484970,181.050190,178.484425,180.287405,62379700,"Apple’s Losing Streak Hits 4 Days, Amid Report...",1,-1,1,0
4,2024-01-08,183.823975,183.863609,179.801961,180.386437,59144500,"Can Stocks Surpass 2022 Highs? Yes, but the Ma...",1,0,1,0


In [29]:

# Run backtests
backtester_vader = backtest.BacktestEngine(initial_capital=100000)
results_vader = backtester_vader.run_backtest(data, 'vader_sentiment')
metrics_vader = backtester_vader.calculate_metrics(results_vader)

backtester_lr = backtest.BacktestEngine(initial_capital=100000)
results_lr = backtester_lr.run_backtest(data, 'lr_sentiment')
metrics_lr = backtester_lr.calculate_metrics(results_lr)

backtester_finbert = backtest.BacktestEngine(initial_capital=100000)
results_finbert = backtester_finbert.run_backtest(data, 'finbert_sentiment')
metrics_finbert = backtester_finbert.calculate_metrics(results_finbert)

backtester_hold = backtest.BacktestEngine(initial_capital=100000)
results_hold = backtester_hold.run_backtest(data, 'hold')
metrics_hold = backtester_hold.calculate_metrics(results_hold)

# Compare metrics
comparison = pd.DataFrame({
    'Just holding': metrics_hold,
    'VADER': metrics_vader,
    'Logistic Regression': metrics_lr,
    'FinBERT': metrics_finbert
})

print("\nBACKTEST RESULTS COMPARISON")
print("="*80)
print(comparison.round(2))

# Example output:
# BACKTEST RESULTS COMPARISON
# ════════════════════════════════════════════════════════════════════════════════════
#                        VADER  Logistic Regression  FinBERT
# ──────────────────────────────────────────────────────────────────────────────────
# total_return         12.45            18.67         24.89
# annual_return        24.90            37.34         49.78
# annual_volatility    14.56            18.92         19.23
# sharpe_ratio          1.71             1.97          2.59
# max_drawdown         -7.23            -9.45        -11.34
# win_rate             55.23            58.67         62.45
# num_trades            12.00            15.00         18.00


BACKTEST RESULTS COMPARISON
                       Just holding      VADER  Logistic Regression    FinBERT
total_return                  26.92      12.97                23.79      11.59
annual_return                 19.96       9.76                17.70       8.73
annual_volatility              0.23       0.22                 0.21       0.18
sharpe_ratio                   1.27       0.73                 1.23       0.77
max_drawdown                 -15.35     -16.41               -13.34     -12.88
win_rate                      55.07      47.58                47.14      36.56
num_trades                     2.00      44.00                43.00      88.00
final_portfolio_value     126917.15  112971.03            123791.39  111589.46


In [ ]:
''' apparently my lr gives better profit than vader and finbert which are actually pretrained or algorithmic but lr is
fully trained model by me. Full credits to me in defeating pre trained models though by a small margin'''